In [14]:
#jupyter notebook needs to be started with prefix KERAS_BACKEND=theano
import theano
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten #Dropout not used yet
from keras.layers import Conv2D, MaxPooling2D #does not know when to use MaxPooling yet
from keras.losses import categorical_crossentropy #loss function normally used for classification problems, some other can be tried
from keras.optimizers import adam #very powerful optimizer (link to an article comparing it to others)
from keras.callbacks import EarlyStopping #to stop the algorithm sooner if the cnn does not improve for some time
                                        #it is inevitable to explain the concept of callback
from keras.callbacks import History #for tracing the loss
from keras.utils import to_categorical #needed for converting labels into a good format

    
import numpy as np
import pandas as pd #for data handling
from sklearn.preprocessing import OneHotEncoder

import matplotlib.pyplot as plt
import sklearn as sk
from sklearn.preprocessing import MinMaxScaler #for MinMaxScaler()

ModuleNotFoundError: No module named 'pandas'

### Data Import 

In [2]:
train = pd.read_csv("train_states.csv",low_memory=False)

# Data Cleaning

In [3]:
print(train.shape)
train = train.drop_duplicates()
print(train.shape) #no duplicates

# The problem is that not all stores have full data available

#this shows how many stores have data available to all the dates
np.sum(train.groupby('Store')['DayOfWeek'].count() != 942) 
#for simplification, let us delete all the stores with incomplete data
#first find out, which store these are
indices = np.where((train.groupby('Store')['DayOfWeek'].count() != 942) == True)
indices = list(indices) #tuple needs to be converted to list
#caution, previous fct. counts from 0, so to get store number we need to add 1
indices = [x+1 for x in indices]
#we need to simplify list containing one array into array
indices = [item for sublist in indices for item in sublist]

#And we can finally filter only the complete stores
train = train[~train['Store'].isin(indices)] #the wave is a negation sign

(1017209, 20)
(1017209, 20)


## Order by Store and Date

In [4]:
train = train.sort_values(by=['Store','Date']) #sort by store and date
len(train[train['Store'] == 1]) #942 dates

942

In [5]:
#Delete unimportant variables
train = train.drop\
        (['is_month_start','is_month_end','is_quarter_start','is_quarter_end','State', #same for all the images
          'week_of_year', #partially identifiable by other variables
          'day_of_year', #identifiable by other variables       
         'day'], axis = 1) #day is partially already in the time dimension

['Store',
 'DayOfWeek',
 'Date',
 'Sales',
 'Customers',
 'Open',
 'Promo',
 'StateHoliday',
 'SchoolHoliday',
 'year',
 'month',
 'quarter']

# Dummies

In [6]:
def dummify(data,data_and_column,prefix):
    dummies = pd.get_dummies(data_and_column, prefix=prefix).iloc[:,1:] #creates dummy variables (12-1 dummy variables with Month_1 as default)
    dataframe = pd.concat([data,dummies], axis = 1)
    return dataframe
#dummies
#train
train = dummify(train,train.DayOfWeek,'DayOfWeek')
train = train.drop('DayOfWeek', axis = 1)
train = dummify(train,train.StateHoliday,'StateHoliday')
train = train.drop('StateHoliday', axis = 1)
train = dummify(train,train.quarter,'Q')  #quarter will also be delete, but later
train = train.drop('quarter', axis = 1)
#we will first change month into months of the quarter 
#(as we will be using quarters later)
#NOT A NICE CHUNK OF CODE
train.month = [1 if e == 4  else e for e in train.month]
train.month = [1 if e == 7  else e for e in train.month]
train.month = [1 if e == 10 else e for e in train.month]
train.month = [2 if e == 5  else e for e in train.month]
train.month = [2 if e == 8  else e for e in train.month]
train.month = [2 if e == 11 else e for e in train.month]
train.month = [3 if e == 6  else e for e in train.month]
train.month = [3 if e == 9  else e for e in train.month]
train.month = [3 if e == 12 else e for e in train.month]
train = dummify(train,train.month,'month')
train = train.drop('month', axis = 1)
#train = train.drop('Date', axis = 1) we need Date for reshaping
train['Date'] = train['Date'].str.replace("-","").astype(int) #convert date to numeric

In [7]:
column_names = pd.DataFrame(list(train))
column_names = column_names.transpose()
np.savetxt("column_names.csv",column_names,fmt="%s",delimiter=",")
#ok now i have a ordered table with the whole time series with each store having 942 rows (days) and 23 variables
#after complete info about first store (942 rows), the second store follows, and so on
#now I need 90 days time series, with 91th day as the label and then move it by 1 day

## Following part creates dataset of 852 time series for 500 stores

In [9]:
timeseries = np.empty([852*90*500,23]) #852, so that last series goes from index 851 to 940 and we have a label on index 941
labels = np.empty([852*500,0])
    
for store in range (0,500): #934
    for i in range (0,852): #for number of stores - 942 because Python does not index the last number
        timeseries[(76680-942)*store+942*store+90*i:(76680-942)*store+942*store+90*i+90,22] = i #to track which ts it it
        timeseries[(76680-942)*store+942*store+90*i:(76680-942)*store+942*store+90*i+90,0:22] = train.iloc[942*store+i:942*store+i+90,:].copy()
        labels[852*store+i] = train.iloc[942*store+i+90,1].copy()
    print(store)
    
    
    

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

In [10]:
timeseries = pd.DataFrame(timeseries)
print("ok1")
labels = pd.DataFrame(labels)
np.savetxt("labels1.csv",labels,delimiter=",")
np.savetxt("timeseries1.csv",timeseries,delimiter=",")
print("ok save")

hotovo ulozit


In [ ]:
timeseries1 = pd.read_csv("timeseries1.csv",low_memory=True,header=None)
column_names = pd.read_csv("column_names.csv",header=None)
column_names[22] = "ts" #index for timeseries needs to be traced for later reshaping
timeseries1.columns = column_names.iloc[0,:]
print("data loaded")

#function which reshapes the data so that each row is one timeseries and columns features for each day
def data_prepare(dataset,ts_number):
    dataset = dataset.set_index(['Store','ts','Date'])
    dataset_reshaped = pd.DataFrame(dataset.values.reshape((ts_number,19*90)))
    return dataset_reshaped

print("function loaded")

data = data_prepare(timeseries1,426000)
print("data prepared")

np.savetxt("final_data1.csv",data,delimiter=",")
print("file saved")

## Creating Train and Test Set 

In [ ]:
test_data = data.sample(n = 42600) #test set is 10% of the whole dataset
test_indexes = test_data.index.values #identify which rows were selected for test set
train_data =  data.drop(test_indexes) #remove rows in the test set from the whole set - create train set
train_data.shape #check if everything's ok

#Save train und test set 
np.savetxt("train_data.csv",train_data,delimiter=",")
print("file saved")
np.savetxt("test_data.csv",test_data,delimiter=",")
print("file saved")

#Creating labels for respective data sets
labels = pd.read_csv("labels1.csv",low_memory=True,header=None) #load labels file created earlier
#using indexes from before, labels for train and test data are extracted from the original file
test_labels = labels.iloc[test_indexes,:] 
train_labels = labels.drop(test_indexes)

#Save train and test set labels
np.savetxt("train_labels.csv",train_labels,delimiter=",")
np.savetxt("test_labels.csv",test_labels,delimiter=",")

# Final CNN Model

In [ ]:
#Data Loading
x_train = pd.read_csv("train_data.csv",sep = ",",header=None, nrows=5000)
print("x_train loaded")
x_train = x_train.values.reshape(383400,90,20,1) #one last reshaping is needed - the 4D tensor is created
print("x_train reshaped")
y_train = pd.read_csv("train_labels.csv",sep = ",",header=None)
y_train = y_train.iloc[0:5000,0]
print("y_train loaded)

x_test = pd.read_csv("test_data.csv",sep = ",",header=None)
x_test = x_test.values.reshape(42600,90,20,1) 
print("x_test loaded")
y_test = pd.read_csv("test_labels.csv",sep = ",",header=None, nrows=500) 
y_test = y_test.iloc[0:500,0]
print("y_test loaded")

In [ ]:
#create model
#Basic variables to be used later
batch_size = 500
epochs = 50
stop_patience = 10 #parameter of early stopping
split = 0.2#validation split par
base_units  = 300
units = base_units
#learning rate is default == 0.01
random.seed(123)

model = Sequential()
model.add(Conv2D(units,                
                 activation = 'relu',           #relu as activation - try also others 
                 kernel_initializer = 'normal', #how the weights should be initialized
                 bias_initializer = 'zeros',    #self-explanatory
                 kernel_size = (7,20),    #7 for one week
                 padding = 'same',  
                 strides = 1,              #how much the conv filter shifts
                 input_shape = (90,20,1)))    #input_shape must be defined based on the input image
              #strides can be added   

model.add(Conv2D(units,                                     
                 activation = 'relu',           #relu as activation - try also others 
                 kernel_initializer = 'normal', #how the weight should be initialized
                 bias_initializer = 'zeros',    #self-explanatory
                 kernel_size = (7,20), 
                 strides = 1,
                 padding = 'same'))

model.add(Flatten()) #transform data from the convolutional layer to format understandable for not conv layer
model.add(Dense(1, activation = 'relu')) #the whole line (parameters) depends on what we are gonna model, needs to be set later

model.compile(optimizer = 'adam',
              loss = 'mean_absolute_error',
              metrics = ['mae']) 
model.fit(x_train, y_train, #these are input data and labels, change according to your data
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,  #shows results stepwise
          validation_split = split,
          callbacks = [EarlyStopping(patience = stop_patience,
                                        restore_best_weights=True)])

In [ ]:
print(model.evaluate(x_test,y_test))
print(model.metrics_names)